In [45]:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
import operator
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

In [4]:
df = pd.read_csv ('weatherHistory.csv')


In [18]:
actual = df[['Temperature (C)']]
predicted = df[['Humidity']]
mse = mean_squared_error(actual, predicted)
rmse = sqrt(mse)
print(rmse)


14.799340006501922


In [19]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
from sklearn.model_selection import KFold # import KFold
kf = KFold(n_splits=10, random_state=2020, shuffle = True)

In [20]:

X_train, X_test, y_train, y_test = train_test_split(actual, predicted, random_state=2020)
Xs_train = scale.fit_transform(X_train)
Xs_test  = scale.transform(X_test)

In [21]:
model = Ridge(alpha=0.1)
model.fit(Xs_train, y_train)
yhat =  model.predict(Xs_test)
residuals = y_test - yhat

In [24]:
mse = mean_squared_error(y_test, yhat)
rmse = sqrt(mse)
print(rmse)

0.15204956992666158


In [70]:
from sklearn.ensemble import RandomForestRegressor as RFR
x = df[['Humidity','Wind Speed (km/h)','Pressure (millibars)','Wind Bearing (degrees)']]
y = df[['Temperature (C)']]
Xtrain,Xtest,ytrain,ytest = tts(x, y, random_state=1693)

In [71]:
rfr = RFR(random_state=1693, n_estimators=100, max_depth=50)
rfr.fit(Xtrain,ytrain)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=50, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1693, verbose=0, warm_start=False)

In [72]:
y_pred = rfr.predict(Xtest)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
print(rmse)

13.593219568028614


In [68]:
x = df[['Humidity','Wind Speed (km/h)','Pressure (millibars)','Wind Bearing (degrees)']]
y= df['Temperature (C)']
xx= x.values
polynomial_features= PolynomialFeatures(degree=6)
x_poly = polynomial_features.fit_transform(xx)
df_poly = pd.DataFrame(data=x_poly)
df_poly

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209
0,1.0,0.89,14.1197,1015.13,251.0,0.7921,12.566533,903.4657,223.39,199.365928,14333.331061,3544.0447,1.030489e+06,254797.63,63001.0,0.704969,11.184214,804.084473,198.8171,177.435676,12756.664644,3154.199783,917135.136041,226769.8907,56070.89,2814.987095,202382.334582,50040.847951,1.455019e+07,3.597666e+06,8.895552e+05,1.046080e+09,2.586527e+08,63954205.13,15813251.0,0.627422,9.953951,715.635181,176.947219,157.917752,...,2.805835e+09,1.334449e+13,3.299546e+12,8.158423e+11,2.017243e+11,4.987815e+10,9.593969e+14,2.372195e+14,5.865465e+13,1.450289e+13,3.585969e+12,8.866631e+11,7.924152e+06,5.697037e+08,1.408643e+08,4.095861e+10,1.012738e+10,2.504086e+09,2.944702e+12,7.281041e+11,1.800303e+11,4.451410e+10,2.117082e+14,5.234674e+13,1.294320e+13,3.200323e+12,7.913085e+11,1.522067e+16,3.763447e+15,9.305461e+14,2.300859e+14,5.689080e+13,1.406676e+13,1.094284e+18,2.705715e+17,6.690123e+16,1.654193e+16,4.090140e+15,1.011324e+15,2.500589e+14
1,1.0,0.86,14.2646,1015.63,259.0,0.7396,12.267556,873.4418,222.74,203.478813,14487.555698,3694.5314,1.031504e+06,263048.17,67081.0,0.636056,10.550098,751.159948,191.5564,174.991779,12459.297900,3177.297004,887093.695334,226221.4262,57689.66,2902.543878,206659.187010,52701.012608,1.471400e+07,3.752277e+06,9.568836e+05,1.047627e+09,2.671596e+08,68129476.03,17373979.0,0.547008,9.073084,645.997555,164.738504,150.492930,...,3.040303e+09,1.305269e+13,3.328621e+12,8.488454e+11,2.164676e+11,5.520229e+10,9.293430e+14,2.369956e+14,6.043723e+13,1.541235e+13,3.930366e+12,1.002299e+12,8.424761e+06,5.998374e+08,1.529670e+08,4.270802e+10,1.089115e+10,2.777397e+09,3.040782e+12,7.754425e+11,1.977488e+11,5.042874e+10,2.165017e+14,5.521099e+13,1.407958e+13,3.590492e+12,9.156263e+11,1.541478e+16,3.930986e+15,1.002457e+15,2.556407e+14,6.519198e+13,1.662488e+13,1.097522e+18,2.798835e+17,7.137426e+16,1.820144e+16,4.641626e+15,1.183680e+15,3.018551e+14
2,1.0,0.89,3.9284,1015.94,204.0,0.7921,3.496276,904.1866,181.56,15.432327,3991.018696,801.3936,1.032134e+06,207251.76,41616.0,0.704969,3.111686,804.726074,161.5884,13.734771,3552.006639,713.240304,918599.334404,184454.0664,37038.24,60.624352,15678.317845,3148.194618,4.054636e+06,8.141678e+05,1.634843e+05,1.048586e+09,2.105554e+08,42279359.04,8489664.0,0.627422,2.769400,716.206206,143.813676,12.223946,...,1.166036e+08,3.724586e+12,7.478940e+11,1.501766e+11,3.015534e+10,6.055171e+09,9.632307e+14,1.934160e+14,3.883779e+13,7.798600e+12,1.565953e+12,3.144422e+11,3.675312e+03,9.504879e+05,1.908573e+05,2.458097e+08,4.935840e+07,9.911129e+06,6.356986e+10,1.276478e+10,2.563159e+09,5.146804e+08,1.644007e+13,3.301154e+12,6.628692e+11,1.331037e+11,2.672711e+10,4.251635e+15,8.537252e+14,1.714274e+14,3.442249e+13,6.912011e+12,1.387927e+12,1.099533e+18,2.207855e+17,4.433356e+16,8.902145e+15,1.787544e+15,3.589376e+14,7.207439e+13
3,1.0,0.83,14.1036,1016.41,269.0,0.6889,11.705988,843.6203,223.27,198.911533,14335.040076,3793.8684,1.033089e+06,273414.29,72361.0,0.571787,9.715970,700.204849,185.3141,165.096572,11898.083263,3148.910772,857464.109123,226933.8607,60059.63,2805.368696,202175.671216,53507.202366,1.457028e+07,3.856126e+06,1.020551e+06,1.050042e+09,2.779010e+08,73548444.01,19465109.0,0.474583,8.064255,581.170025,153.810703,137.030155,...,3.213623e+09,1.249349e+13,3.306489e+12,8.750855e+11,2.315975e+11,6.129389e+10,9.003736e+14,2.382901e+14,6.306515e+13,1.669063e+13,4.417292e+12,1.169067e+12,7.870094e+06,5.671773e+08,1.501074e+08,4.087500e+10,1.081785e+10,2.863021e+09,2.945756e+12,7.796148e+11,2.063305e+11,5.460681e+10,2.122930e+14,5.618482e+13,1.486971e+13,3.935371e+12,1.041524e+12,1.529941e+16,4.049095e+15,1.071621e+15,2.836121e+14,7.505991e+13,1.986513e+13,1.102589e+18,2.918078e+17,7.722898e+16,2.043919e+16,5.409374

In [69]:
model = LinearRegression()
model.fit(x_poly, y.values)
y_poly_pred = model.predict(x_poly)
rmse = np.sqrt(mean_squared_error(y,y_poly_pred))
rmse

6.069699316489059

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)
Xs_train = scale.fit_transform(X_train)
Xs_test  = scale.transform(X_test)

In [63]:
def DoKFold(model,X,y,k,random_state=1234,scaler=None):
    '''Function will perform K-fold validation and return a list of K training and testing scores, inclduing R^2 as well as MSE.
    
        Inputs:
            model: An sklearn model with defined 'fit' and 'score' methods
            X: An N by p array containing the features of the model. The N rows are observations, and the p columns are features.
            y: An array of length N containing the target of the model
            k: The number of folds to split the data into for K-fold validation
            random_state: used when splitting the data into the K folds (default=146)
            scaler: An sklearn feature scaler.  If none is passed, no feature scaling will be performed
        Outputs:
            train_scores: A list of length K containing the training scores
            test_scores: A list of length K containing the testing scores
            train_mse: A list of length K containing the MSE on training data
            test_mse: A list of length K containing the MSE on testing data
    '''
    
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=k,shuffle=True,random_state=random_state)
    
    train_scores=[]
    test_scores=[]
    train_mse=[]
    test_mse=[]
    
    for idxTrain, idxTest in kf.split(X):
        Xtrain = X[idxTrain,:]
        Xtest = X[idxTest,:]
        ytrain = y[idxTrain]
        ytest = y[idxTest]
        
        if scaler != None:
            Xtrain = scaler.fit_transform(Xtrain)
            Xtest = scaler.transform(Xtest)
        
        model.fit(Xtrain,ytrain)
        
        train_scores.append(model.score(Xtrain,ytrain))
        test_scores.append(model.score(Xtest,ytest))
        
        # Compute the mean squared errors
        ytrain_pred = model.predict(Xtrain)
        ytest_pred = model.predict(Xtest)
        train_mse.append(np.mean((ytrain-ytrain_pred)**2))
        test_mse.append(np.mean((ytest-ytest_pred)**2))
        
    return train_scores,test_scores,train_mse,test_mse

In [64]:
x= df['Temperature (C)'].to_numpy()
y=df['Humidity'].to_numpy()
DoKFold(Ridge,x,y,10,random_state=1234,scaler=StandardScaler)

IndexError: ignored